### TwoAgenChat with coding

This notebook shows how to use a `NotebookUserAgent` and a `GPTAgent` to implement a back-and-force coding-debuging conversation.

### Install dependencies

In [9]:
// #i "nuget:https://www.myget.org/F/agentchat/api/v3/index.json"

#r "nuget:AgentChat.Core,*-*"
#r "nuget:AgentChat.GPT,*-*"
#r "nuget:AgentChat.Example.Share,*-*"
// this package provides FunctionDefinition for `dotnet-interactive` command
#r "nuget:AgentChat.DotnetInteractiveFunction,*-*"

Installed Packages AgentChat.Core, 0.2.1-1 AgentChat.DotnetInteractiveFunction, 0.2.1-1 AgentChat.Example.Share, 0.2.1-1 AgentChat.GPT, 0.2.1-1

### Step 1: Create a NotebookUserAgent
The first step we are going to do is to define a `NotebookUserAgent` that will be used to take input from user and send it to the coding agent.

In [10]:
using AgentChat.Example.Share;
using AgentChat;

IAgent user = new NotebookUserAgent("User");

### Step 2: Create a Coding Agent
Then we are going to create a `GPTAgent` that will be used to write dotnet script code to resolve the user's problem.

In [11]:
var coder = Constant.GPT35.CreateAgent(
    name: "Coder",
    roleInformation: @"You act as dotnet coder, you write dotnet script to resolve task.
-workflow-
write_code_to_resolve_coding_task

if code_has_error
    fix_code_error

if task_complete, say [COMPLETE]

-end-

Here're some rules to follow on write_code_to_resolve_current_step:
- put code between ```csharp and ```
- Use top-level statements, remove main function, just write code, like what python does.
- Remove all `using` statement. Runner can't handle it.
- Try to use `var` instead of explicit type.
- Try avoid using external library.
- Don't use external data source, like file, database, etc. Create a dummy dataset if you need.
- Always print out the result to console. Don't write code that doesn't print out anything.

Here are some examples for write_code_to_resolve_coding_task:
```nuget
xxx
```
```csharp
xxx
```

Here are some examples for fix_code_error:
The error is caused by xxx. Here's the fix code
```csharp
xxx
```",
    temperature: 0);

### Step 3: Create a runner agent to run code and get output
In this step, we are going to create another `GPTAgent` that will be used to run the code and get the output.
The runner agent will use `function_call` to call `RunDotnetCodeFunction` and `InstallNugetDependencyFunction` to run the code and get the output.

In [12]:
using System.IO;
using AgentChat.DotnetInteractiveService;
using Azure.AI.OpenAI;

var workDir = Path.Combine(Path.GetTempPath(), "InteractiveService");
if (!Directory.Exists(workDir))
    Directory.CreateDirectory(workDir);

var service = new InteractiveService(workDir);
var dotnetInteractiveFunctions = new DotnetInteractiveFunction(service);

// this function is used to fix invalid json returned by GPT-3
var fixInvalidJsonFunction = new FixInvalidJsonFunctionWrapper(Constant.GPT35);

var runner = Constant.GPT35.CreateAgent(
    name: "Runner",
    roleInformation: @"you act as dotnet runner, you run dotnet script and install nuget packages. Here's the workflow you follow:
-workflow-
if code_is_available
    call run_code

if nuget_packages_is_available
    call install_nuget_packages

for any other case
    say [NO_CODE_AVAILABLE]
-end-",
    temperature: 0,
    functionMap: new Dictionary<FunctionDefinition, Func<string, Task<string>>> {
        { dotnetInteractiveFunctions.RunCodeFunction, fixInvalidJsonFunction.FixInvalidJsonWrapper(dotnetInteractiveFunctions.RunCodeWrapper) },
        { dotnetInteractiveFunctions.InstallNugetPackagesFunction, dotnetInteractiveFunctions.InstallNugetPackagesWrapper },
    });

// start kenel
await service.StartAsync(workDir, default);

### Play with runner agent
Now we can play with the runner agent to see how it works.

In [13]:
// runner should respond with NO_CODE_AVAILABLE
var greeting = new Message(ChatRole.User, "Hello", from: user.Name);
await user.SendMessageToAgentAsync(
    runner,
    new [] {greeting},
    maxRound: 1);

// runner should run code
var someCodeMessage = new Message(ChatRole.User, @"Console.WriteLine(""Hello World!!!"")", from: user.Name);
await user.SendMessageToAgentAsync(
    runner,
    new [] {someCodeMessage},
    maxRound: 1);

// runner should install nuget packages
var installNugetPackagesMessage = new Message(ChatRole.User, @"nuget install Microsoft.ML", from: user.Name);

await user.SendMessageToAgentAsync(
    runner,
    new [] {installNugetPackagesMessage},
    maxRound: 1);

Message from Runner
--------------------
NO_CODE_AVAILABLE

--------------------

function name: RunCode
raw function call: {
  "code": "Console.WriteLine(\"Hello World!!!\")"
}
Message from Runner
--------------------
Hello World!!!

--------------------

function name: InstallNugetPackages
raw function call: {
  "nugetPackages": ["Microsoft.ML"]
}
Message from Runner
--------------------
Installed nuget packages:
- Microsoft.ML

--------------------



### Step 4: Create two agents chat

In this step, we are going to create a `TwoAgentChat` to let the `user` agent and the `coder` agent chat with each other.

The Coding agent will take the user's input and write dotnet script code to resolve the user's problem.

And once the code is ready, the Coding agent will send the code to the User agent.

Then the User agent will send the code to the Runner agent to run the code and get the output, then send the output back to the Coder agent.

If the output is correct, the Coder agent will return a 'COMPLETE' message back to User agent and wait for next input. Otherwise, the Coder agent will try to fix the code and send the code back to the User agent to run again.


In [15]:
// use runner agent to auto-reply message from coder
user = user.WithAutoReply(async (msgs) => {
    // if last message contains "COMPLETE", stop sending messages to runner agent and fall back to user agent
    if (msgs.Last().Content.Contains("COMPLETE"))
        return null; // return null to fall back to user agent
    
    // otherwise, send message to runner agent to either run code or install nuget packages and get the reply
    var reply = await runner.SendMessageAsync(msgs);
    // the from field must be set to user's name
    var message = new Message(ChatRole.Assistant, reply.Content, from: user.Name);
    return message;
});

var codingTask = new Message(ChatRole.User, "what's the 10th of fibonacci? Print the question and result in the end.", from: user.Name);

await user.SendMessageToAgentAsync(
    coder,
    new [] {codingTask},
    maxRound: 10);

Message from Coder
--------------------
```csharp
int n = 10;
int fib1 = 0;
int fib2 = 1;
int fibN = 0;

for (int i = 2; i <= n; i++)
{
    fibN = fib1 + fib2;
    fib1 = fib2;
    fib2 = fibN;
}

Console.WriteLine($"The 10th Fibonacci number is: {fibN}");
```
--------------------

function name: RunCode
raw function call: {
  "code": "int n = 10;\nint fib1 = 0;\nint fib2 = 1;\nint fibN = 0;\n\nfor (int i = 2; i <= n; i++)\n{\n    fibN = fib1 + fib2;\n    fib1 = fib2;\n    fib2 = fibN;\n}\n\nConsole.WriteLine($\"The 10th Fibonacci number is: {fibN}\");"
}
Runner: The 10th Fibonacci number is: 55

Message from User
--------------------
The 10th Fibonacci number is: 55

--------------------

Message from Coder
--------------------
COMPLETE
--------------------

Message from User
--------------------
what's the 20th Fibonacci?
--------------------

Message from Coder
--------------------
```csharp
int n = 20;
int fib1 = 0;
int fib2 = 1;
int fibN = 0;

for (int i = 2; i <= n; i++)
{
    fi

Error: Input request cancelled

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 72
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at AgentChat.Example.Share.NotebookUserAgent.CallAsync(IEnumerable`1 messages, CancellationToken ct)
   at AgentChat.AutoReplyAgent.CallAsync(IEnumerable`1 conversation, CancellationToken ct)
   at AgentChat.SequentialGroupChat.CallAsync(IEnumerable`1 conversationWithName, Int32 maxRound, Boolean throwExceptionWhenMaxRoundReached)
   at AgentChat.AgentExtension.SendMessageToAgentAsync(IAgent agent, IAgent receiver, IEnumerable`1 chatHistory, Int32 maxRound, Boolean throwWhenMaxRoundReached, CancellationToken ct)
   at Submission#16.<<Initialize>>d__0.